In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("./data/train.csv")

In [3]:
# Look at the top 5 data
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Each row corresponds to one training example and the label and pixel value of each example

In [5]:
# Convert pandas dataframe to numpy array to use linear algebra
data = np.array(data)
# get the dimensions of the data
m, n = data.shape
# Shuffle the dataset
np.random.shuffle(data)

In [46]:
m

42000

In [6]:
# Create train-test split

# Validation data
val_data = data[:1000].T
Y_val = val_data[0]
X_val = val_data[1: n]

# Train data
train_data = data[1000: m].T
Y_train = train_data[0]
X_train = train_data[1: n]


In [49]:
print(Y_train)
print(X_train[:, 0].shape)

[9 1 2 ... 6 1 7]
(784,)


In [11]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [12]:
# Rectified Linear Unit function (Activation function)
def ReLU(Z):
    # Takes every single element in Z, if it's more than 0, it will return the element else, it will return 0
    return np.maximum(0, Z)

In [22]:
def deriv_ReLU(Z):
    return Z > 0

In [18]:
# SoftMax Function (Activation function)
def softMax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

In [30]:
def forward_prop(W1, b1, W2, b2, X):

    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softMax(A1)
    return Z1, A1, Z2, A2
    

In [21]:
# One hot encoding for Y labels
def one_hot(Y):
    # Y.size is essentially m
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # Creates a null matrix with the size of the amount of labels
    one_hot_Y[np.arange(Y.size), Y] = 1 # Goes to the row/column of the label and set the value at that index to 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [38]:
def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)

    dZ2 = A2 - one_hot_Y # Error of output layer
    dW2 = 1 / m * dZ2.dot(A1.T) # Derivative of Loss function with respect to weight
    db2 = 1/m * np.sum(dZ2) # Average of absolute error

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1) # Error of Hidden layer
    dW1 = 1 / m * dZ1.dot(X.T) # Derivative of Loss function with respect to weight
    db1 = 1/m * np.sum(dZ1)  # Average of absolute error

    return dW1, db1, dW2, db2


In [24]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha* db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha* db2

    return W1, b1, W2, b2

In [36]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def get_predictions(A2):
    return np.argmax(A2, 0)

In [44]:
def gradient_descent(x, Y, iter, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iter):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, x)
        dW1 , db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, x, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 50) == 0:
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_predictions(A2), Y)}")
        
    return W1, b1, W2, b2

In [45]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 1000, 0.1)

<ipython-input-18-8877bab4ad09>:3: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
<ipython-input-18-8877bab4ad09>:3: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(Z) / np.sum(np.exp(Z))


Iteration: 0
[0 1 5 ... 9 7 0] [9 1 2 ... 6 1 7]
Accuracy: 0.10558536585365853
Iteration: 50
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 100
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 150
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 200
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 250
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 300
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 350
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 400
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 450
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 500
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 550
[0 0 0 ... 0 0 0] [9 1 2 ... 6 1 7]
Accuracy: 0.09851219512195122
Iteration: 600
[0 0 0 ... 0 0 0